In [247]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [248]:
df=pd.read_csv('D:\\train_file.csv')
df.head()

,Patient_ID,YEAR,LocationDesc,Subtopic,Greater_Risk_Question,Description,Sample_Size,Sex,Race,Grade,GeoLocation,QuestionCode,StratID1,StratID2,StratID3,StratificationType,Greater_Risk_Probability
0,1,1995,"Houston, TX",1,Ever used cocaine,"any form of cocaine, such as powder, crack, or...",335.0,Female,Hispanic or Latino,4,"(29.760427, -95.369803)",H49,1,4,0,Local,7.4793
1,2,1997,"Houston, TX",0,Currently drank alcohol,"at least one drink of alcohol, on at least 1 d...",382.0,Total,Black or African American,4,"(29.760427, -95.369803)",H42,0,3,0,Local,49.6852
2,3,1993,Nevada,0,Had their first drink of alcohol before age 13...,other than a few sips,196.0,Female,White,0,"(39.493240390000494, -117.07184056399967)",H41,1,6,2,State,47.8132
3,4,2017,Pennsylvania,1,Ever used cocaine,"any form of cocaine, such as powder, crack, or...",104.0,Male,Asian,4,"(40.79373015200048, -77.86070029399963)",H49,2,2,0,State,3.4554
4,5,2007,"San Francisco, CA",1,Ever took steroids without a doctor's prescrip...,"pills or shots, one or more times during their...",100.0,Female,Multiple Race,4,"(37.77493, -122.419416)",H55,1,7,0,Local,0.0000


In [249]:
#df['GeoLocation_from']=df['GeoLocation_from'].fillna('40.712784')
#df['GeoLocation_to']=df['GeoLocation_to'].fillna('74.005941')
df['GeoLocation_from']=df['GeoLocation'].str.findall('(\d+)').str[0]
df['GeoLocation_to']=df['GeoLocation'].str.findall('(\d+)').str[-2]
df['GeoLocation_from']=df['GeoLocation_from'].astype('float')
df['GeoLocation_to']=df['GeoLocation_to'].astype('float')
df=df.drop(['GeoLocation','LocationDesc','YEAR'],axis=1)

In [250]:
df.isnull().sum()

Patient_ID                     0
Subtopic                       0
Greater_Risk_Question          0
Description                    0
Sample_Size                    0
Sex                            0
Race                           0
Grade                          0
QuestionCode                   0
StratID1                       0
StratID2                       0
StratID3                       0
StratificationType             0
Greater_Risk_Probability       0
GeoLocation_from            3301
GeoLocation_to              3301
dtype: int64

In [251]:
le=LabelEncoder()
df['Sex']=le.fit_transform(df['Sex'])
df['Sex']=df['Sex'].astype('category')
df['Race']=le.fit_transform(df['Race'])
df['Race']=df['Race'].astype('category')
df['StratificationType']=le.fit_transform(df['StratificationType'])
df['StratificationType']=df['StratificationType'].astype('category')

In [252]:
df.head()

,Patient_ID,Subtopic,Greater_Risk_Question,Description,Sample_Size,Sex,Race,Grade,QuestionCode,StratID1,StratID2,StratID3,StratificationType,Greater_Risk_Probability,GeoLocation_from,GeoLocation_to
0,1,1,Ever used cocaine,"any form of cocaine, such as powder, crack, or...",335.0,0,3,4,H49,1,4,0,0,7.4793,29.0,95.0
1,2,0,Currently drank alcohol,"at least one drink of alcohol, on at least 1 d...",382.0,2,2,4,H42,0,3,0,0,49.6852,29.0,95.0
2,3,0,Had their first drink of alcohol before age 13...,other than a few sips,196.0,0,7,0,H41,1,6,2,3,47.8132,39.0,117.0
3,4,1,Ever used cocaine,"any form of cocaine, such as powder, crack, or...",104.0,1,1,4,H49,2,2,0,3,3.4554,40.0,77.0
4,5,1,Ever took steroids without a doctor's prescrip...,"pills or shots, one or more times during their...",100.0,0,4,4,H55,1,7,0,0,0.0000,37.0,122.0


In [253]:
df['QuestionCode']=df['QuestionCode'].str.split('H').str[1]
df['QuestionCode']=df['QuestionCode'].str.replace('ALLUCDRUG','48')
df['QuestionCode']=df['QuestionCode'].astype('float')

In [254]:
df['GeoLocation_from']=df['GeoLocation_from'].fillna(40.0)
df['GeoLocation_to']=df['GeoLocation_to'].fillna(80.0)

In [255]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [256]:
tf=TfidfVectorizer(stop_words='english',analyzer='char')
x=tf.fit_transform(df['Greater_Risk_Question'])
df['Greater_Risk_Question']=x.toarray()
y=tf.fit_transform(df['Description'])
df['Description']=y.toarray()

In [257]:
df.head()

,Patient_ID,Subtopic,Greater_Risk_Question,Description,Sample_Size,Sex,Race,Grade,QuestionCode,StratID1,StratID2,StratID3,StratificationType,Greater_Risk_Probability,GeoLocation_from,GeoLocation_to
0,1,1,0.313062,0.592818,335.0,0,3,4,49.0,1,4,0,0,7.4793,29.0,95.0
1,2,0,0.227606,0.660636,382.0,2,2,4,42.0,0,3,0,0,49.6852,29.0,95.0
2,3,0,0.492286,0.573609,196.0,0,7,0,41.0,1,6,2,3,47.8132,39.0,117.0
3,4,1,0.313062,0.592818,104.0,1,1,4,49.0,2,2,0,3,3.4554,40.0,77.0
4,5,1,0.312752,0.572084,100.0,0,4,4,55.0,1,7,0,0,0.0000,37.0,122.0


In [258]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
x=df.drop(['Patient_ID','Greater_Risk_Probability'],axis=1)
y=df['Greater_Risk_Probability']
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)

In [259]:
leg=GradientBoostingRegressor(n_estimators=150)
leg.fit(x_train,y_train)
y_leg=leg.predict(x_test)
np.sqrt(mean_squared_error(y_leg,y_test))

7.218352843744093

In [260]:
test=pd.read_csv('D:\\test_file.csv')
ide=test['Patient_ID']
test=test.drop(['Patient_ID','YEAR','LocationDesc'],axis=1)
test.shape

(36932, 13)

In [261]:
#test['GeoLocation_from']=test['GeoLocation_from'].fillna('40.712784')
#test['GeoLocation_to']=test['GeoLocation_to'].fillna('74.005941')
test['GeoLocation_from']=test['GeoLocation'].str.findall('(\d+)').str[0]
test['GeoLocation_to']=test['GeoLocation'].str.findall('(\d+)').str[-2]
test['GeoLocation_from']=test['GeoLocation_from'].astype('float')
test['GeoLocation_to']=test['GeoLocation_to'].astype('float')
test=test.drop(['GeoLocation'],axis=1)

In [262]:
le=LabelEncoder()
test['Sex']=le.fit_transform(test['Sex'])
test['Sex']=test['Sex'].astype('category')
test['Race']=le.fit_transform(test['Race'])
test['Race']=test['Race'].astype('category')
test['StratificationType']=le.fit_transform(test['StratificationType'])
test['StratificationType']=test['StratificationType'].astype('category')

In [263]:
tf=TfidfVectorizer(stop_words='english',analyzer='char')
x=tf.fit_transform(test['Greater_Risk_Question'])
test['Greater_Risk_Question']=x.toarray()
y=tf.fit_transform(test['Description'])
test['Description']=y.toarray()

In [264]:
test['QuestionCode']=test['QuestionCode'].str.split('H').str[1]
test['QuestionCode']=test['QuestionCode'].str.replace('ALLUCDRUG','48')
test['QuestionCode']=test['QuestionCode'].astype('float')

In [265]:
test['GeoLocation_from']=test['GeoLocation_from'].fillna(40.0)
test['GeoLocation_to']=test['GeoLocation_to'].fillna(80.0)

In [266]:
y_test=leg.predict(test)
#y_test=y_test.astype('int')

In [267]:
res=pd.DataFrame(y_test)
res.columns=['Greater_Risk_Probability']
res['Patient_ID']=ide
res=res[['Patient_ID','Greater_Risk_Probability']]

In [268]:
res.to_csv('D:\\answer.csv',index=False)
res.shape

(36932, 2)

In [269]:
a=pd.read_csv('D:\\answer.csv')

In [270]:
b=np.arange(212114,249046,1)

In [271]:
a.head()
a['Patient_ID']=b

In [272]:
a.to_csv('D:\\answer2.csv',index=False)

In [273]:
a

,Patient_ID,Greater_Risk_Probability
0,212114,46.635824
1,212115,25.609493
2,212116,42.638228
3,212117,13.958790
4,212118,8.763356
5,212119,38.631626
6,212120,10.390437
7,212121,4.872747
8,212122,19.981957
9,212123,7.751799
